In [1]:
"""
To retrieve data from the Tripadvisor forum, we must generate a series of URLs based on the pattern observed in the 
pagination extensions. For the Hong Kong forum (g294217-i1496), the links follow this structure:

ShowForum-g294217-i1496-Hong_Kong.html (first page)
ShowForum-g294217-i1496-o20-Hong_Kong.html (page 2)
ShowForum-g294217-i1496-o40-Hong_Kong.html (page 3)
ShowForum-g294217-i1496-o60-Hong_Kong.html (page 4)
and so forth.

One can clearly see that, starting from the second page, the suffix increments by 20 each time in the form “-oXX”, 
where XX increases in steps of 20 (i.e., o20, o40, o60, o80, …).In order to scrape a comprehensive range of data, 
we need to generate an array (or list) of these extensions programmatically.
"""
lower_bound = 39
upper_bound = 50
ShowForum = []
while lower_bound < upper_bound:
    page_index = "-o" + str(20*lower_bound)
    ShowForum_url = f"https://www.tripadvisor.com/ShowForum-g294217-i1496{page_index}-Hong_Kong.html"
    ShowForum.append(ShowForum_url)
    lower_bound = lower_bound + 1
ShowForum

['https://www.tripadvisor.com/ShowForum-g294217-i1496-o780-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o800-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o820-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o840-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o860-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o880-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o900-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o920-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o940-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o960-Hong_Kong.html',
 'https://www.tripadvisor.com/ShowForum-g294217-i1496-o980-Hong_Kong.html']

In [4]:
'''
The following pair of functions has been implemented to start and stop the Surfshark VPN service using its native 
Linux command-line interface. Since the Surfshark client operates at sudo level and will invariably request the 
administrative password, the solution utilises Pexpect to handle the prompt programmatically, inserting the requisite 
password that has been securely predefined for this purpose.
'''
import pexpect
import sys
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
from webdriver_manager.chrome import ChromeDriverManager
import time
password = "davidisdumb" # Hardcoded Sudo password

def StartVPN():
    # i = random.randint(0, 140) # Randomly selects a Surfshark server location from the available list
    print("Starting VPN")
    command = "sudo surfshark-vpn attack"
    
    # Spawn the process
    child = pexpect.spawn(command, encoding='utf-8')

    # Expect the password prompt
    child.expect('.*password.*:')
        
    # Send the password
    child.sendline(password)

    #child.expect("press enter for next page")
    #child.sendline('ls -l')
    #child.sendline('ls -l')
    #child.sendline('ls -l')
    #child.sendline('ls -l')
    #child.sendline('ls -l')
    #child.expect("Enter a number to select the location")   
    #child.sendline(str(i))
    time.sleep(1)
    # Expect the  prompt
    child.expect("Enter a number to select the VPN connection type. For default UDP, press ENTER")
    
    child.sendline('ls -l')

    # Capture output until the process completes
    child.expect(pexpect.EOF)
        
    # Print the output
    print(child.before)
    print("---------------------------------------------------------------")
    child.terminate(force=True)

def DownVPN():
    command = "sudo surfshark-vpn down"
    # Spawn the process
    child = pexpect.spawn(command, encoding='utf-8')
    # Expect the password prompt
    child.expect('.*password.*:')       
    # Send the password
    child.sendline(password)
    # Capture output until the process completes
    child.expect(pexpect.EOF)
    # Print the output
    print(child.before)
    child.terminate(force=True)


chrome_options = Options()
chrome_options.add_argument("--headless=new")  # or remove for visible browser
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
# Optional: spoof a real user profile
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")
chrome_options.add_argument("--enable-javascript")  # Ensure JavaScript is enabled
chrome_options.add_argument("--enable-cookies")  # Ensure cookies are enabled
chrome_options.add_argument("--blink-settings=imagesEnabled=true")  # Ensure images load
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])  # Disable automation flags
chrome_options.add_experimental_option('useAutomationExtension', False)  # Disable automation extension

def HTML_get(target,HTML_name):
    DownVPN()
    # Set up ChromeDriver service
    service = Service(ChromeDriverManager().install())

    # Initialize WebDriver
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # === APPLY STEALTH ===
    stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",           # or "NVIDIA Corporation"
        renderer="Intel Iris OpenGL Engine", # matches real devices
        fix_hairline=True,                   # fixes 1px hairline bug in headless
        hide_webdriver=True,                 # sets navigator.webdriver = undefined
        hide_automation=True,                # removes AutomationControlled blink feature
        exclude_switches_to_hide=["enable-automation", "enable-logging"],
        exclude_cdc=True,                    # removes $cdc_ variables
        )
    '''
    Examine the cookie yield. A properly cooperative site should furnish us with approximately 18–20 cookies. 
    Anything markedly below this threshold strongly suggests that we’ve been quietly shown the door by Cloudflare 
    (or similar) and are, regrettably, blocked.
    '''
    
    number_of_cookies = 0
    html_source = None
    while number_of_cookies < 4: 
        StartVPN()
        driver.get(target)
        time.sleep(1)
        html_source = driver.page_source
        cookies = driver.get_cookies()
        number_of_cookies = len(cookies)
        if number_of_cookies > 4:
            print(f"Cookies captured: {len(cookies)}")
            # Save to file
            with open(HTML_name, "w", encoding="utf-8") as f:
                f.write(html_source)
        DownVPN()
    driver.quit()
   


In [5]:
for i in range(len(ShowForum)):
    print(ShowForum[i])
    HTML_get(ShowForum[i],str(i) + ".html")

https://www.tripadvisor.com/ShowForum-g294217-i1496-o780-Hong_Kong.html
 
VPN is down.

Starting VPN

0 TCP
1 UDP
1 selected
ls -l
Checking for connection status...
Connected to Surfshark VPN
Your new IP: 138.199.62.8
Received/Sent: 34 kB/22 kB
Received/Sent: 34 kB/22 kB

---------------------------------------------------------------
Cookies captured: 19
 
VPN is down.

https://www.tripadvisor.com/ShowForum-g294217-i1496-o800-Hong_Kong.html
 
VPN is down.

Starting VPN

0 TCP
1 UDP
1 selected
ls -l
Checking for connection status...
Connected to Surfshark VPN
Your new IP: 138.199.62.153
Received/Sent: 46 kB/36 kB
Received/Sent: 46 kB/36 kB

---------------------------------------------------------------
Cookies captured: 23
 
VPN is down.

https://www.tripadvisor.com/ShowForum-g294217-i1496-o820-Hong_Kong.html
 
VPN is down.

Starting VPN

0 TCP
1 UDP
1 selected
ls -l
Checking for connection status...
Connected to Surfshark VPN
Your new IP: 156.146.45.199
Received/Sent: 29 kB/26 kB
Rec

In [34]:
"""
Extract every single topic from the forum
"""
ShowForum_number = len(ShowForum)
from bs4 import BeautifulSoup
showtopics = []
for i in range(ShowForum_number):
    html_file_path = f"{str(i)}.html"
    print("----------------------------------------------")
    print(f"ShowTopic found in {html_file_path}")
    print("----------------------------------------------")
    # Read the local HTML file
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    #Find all <a> tags that are direct children of <b> tags
    bold_links = soup.find_all('b')                  # find every <b>
    n = 0
    for b in bold_links:
        # Look for <a> tags directly inside this <b>
        a_tag = b.find('a', href=True)               # first <a> with href inside this <b>
        if a_tag:                                    # if exists
            n = n + 1
            showtopic = a_tag['href']
            showtopics.append(showtopic)
            print(f" {str(n)}<-----{showtopic}")


----------------------------------------------
ShowTopic found in 0.html
----------------------------------------------
 1<-----/ShowTopic-g294217-i1496-k14870207-Happy_Valley_race_day_packages-Hong_Kong.html
 2<-----/ShowTopic-g294217-i1496-k14819843-Hong_Kong_Horse_Racing-Hong_Kong.html
 3<-----/ShowTopic-g294217-i1496-k14869107-Help_Tips_for_my_16_full_days_in_Hong_Kong-Hong_Kong.html
 4<-----/ShowTopic-g294217-i1496-k14861434-Best_Luxury_hotel-Hong_Kong.html
 5<-----/ShowTopic-g294217-i1496-k14868835-HKDL_hotel_to_Tsim_sha_Tsui_with_Taxi_airport_trf_from_TST-Hong_Kong.html
 6<-----/ShowTopic-g294217-i1496-k14866675-Hong_Kong_Airport_8_hour_layover-Hong_Kong.html
 7<-----/ShowTopic-g294217-i1496-k14863216-URGENT_Travel_plan_change_due_to_Typhoon_affecting_Japan-Hong_Kong.html
 8<-----/ShowTopic-g294217-i1496-k14861711-Overnight_stay_in_HK-Hong_Kong.html
 9<-----/ShowTopic-g294217-i1496-k14864330-PAR_for_Indian_Passport_Holders-Hong_Kong.html
 10<-----/ShowTopic-g294217-i1496-k148633

In [35]:
print(len(showtopics))

220
